# Mount drive and append path to PYTONPATH


In [ ]:
import gc
import os
import sys

import colab_functions
import colab_utils
import pandas as pd
import prepare_data
import pytorch_lightning as pl
import timm
import train_NN
from google.colab import drive
from lightning.pytorch.loggers import CSVLogger

drive.mount("/content/drive")
sys.path.append("/content/drive/MyDrive/DeepLCMS/train_google_colab")

In [ ]:
%%capture
!pip install lightning
!pip install timm
!pip install torchinfo
!pip install lets-plot


# Import and install libraries

In [ ]:

%%capture
!pip install lightning
!pip install timm
!pip install torchinfo
!pip install lets-plot

In [ ]:
# Set the CUDA_VISIBLE_DEVICES environment variable
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Unzip data

In [ ]:
!unzip -q experiment.zip

# Check if GPU is used

In [ ]:
device = colab_functions.get_device()

# Taking a look at the list of Timm pretrained models

In [ ]:
timm_model_db = pd.read_csv(
    "https://raw.githubusercontent.com/huggingface/pytorch-image-models/main/results/results-imagenet.csv"
)

In [ ]:
# Most common unique architecture families

most_common = (
    timm_model_db.model.str.split("_", expand=True)[0]
    .str.split(".", expand=True)[0]
    .str.split("[0-9]", regex=True, expand=True)[0]
    .value_counts()
    .sort_values(ascending=False)
    .head(20)
)
most_common

In [ ]:
most_common_least_parameters = []

for most_common_one in most_common.index:
    try:
        _ = (
            timm_model_db.assign(
                param_count=lambda df: df.param_count.str.replace(",", "").astype(float)
            )
            .query("model.str.contains(@most_common_one) and 10<param_count")
            .sort_values(by="param_count")
            .reset_index(drop=True)
            .loc[0, ["model", "param_count"]]
            .to_dict()
        )
        most_common_least_parameters.append(_)
    except KeyError:
        pass

most_common_least_parameters_df = pd.DataFrame(most_common_least_parameters)
most_common_least_parameters_df

# Findings the best architecture families based on the models with least


In [ ]:
%%script echo skipping

from typing import Optional, Tuple

import colab_utils
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchinfo
from pytorch_lightning import LightningModule
from pytorch_lightning.callbacks import Callback
from pytorch_lightning.trainer.trainer import Trainer
from timm import create_model
from torchmetrics import Accuracy
from torchmetrics.classification import BinaryF1Score

PRETRAINED_MODEL= "tf_efficientnetv2_b2.in1k"
class ExampleModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = timm.create_model(PRETRAINED_MODEL, pretrained=True, num_classes=1)

        # Freeze all layers except for the last one
        for param in self.model.parameters():
            param.requires_grad = False
        number_of_features_in = int(self.model.classifier.in_features)

        self.model.classifier = torch.nn.Sequential(
            torch.nn.Linear(in_features=number_of_features_in,
                            out_features=int(number_of_features_in/2), bias=True),
            torch.nn.ReLU(),
            torch.nn.Dropout(p=0.3),
            torch.nn.Linear(in_features=int(number_of_features_in/2),
                            out_features=int(number_of_features_in/4), bias=True),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=int(number_of_features_in/4),
                            out_features=1, bias=True),
        )

    def forward(self, x):
        x = self.model(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch

        loss_fn = nn.BCELoss()

        y_pred_logits = self(x).squeeze()
        y_pred = torch.sigmoid(y_pred_logits)
        loss = loss_fn(y_pred, y.float())

        # Calculate metrics
        y_pred_class = torch.round(y_pred)
        acc = (y_pred_class == y).sum().item() / len(y_pred)

        metric_f1 = BinaryF1Score().to(y.device)
        f1 = metric_f1(y_pred_class, y)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch

        loss_fn = nn.BCELoss()

        y_pred_logits = self(x).squeeze()
        y_pred = torch.sigmoid(y_pred_logits)
        loss = loss_fn(y_pred, y.float())
        self.log(
            "val_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True
        )

        # Calculate metrics
        y_pred_class = torch.round(y_pred)
        acc = (y_pred_class == y).sum().item() / len(y_pred)
        self.log(
            "val_acc", acc, on_step=False, on_epoch=True, prog_bar=True, logger=True
        )

        metric_f1 = BinaryF1Score().to(y.device)
        f1 = metric_f1(y_pred_class, y)
        self.log(
            "val_f1", f1, on_step=False, on_epoch=True, prog_bar=True, logger=True
        )

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        if isinstance(batch, list):
            # Assuming the first element in the list is the input tensor
            input_tensor = batch[0]
            return self(input_tensor)
        else:
            # If batch is already a tensor, proceed as usual
            print("Input Shape:", batch.shape)
            return self(batch)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(
            self.parameters(),
            lr=0.001,
            weight_decay=2e-5,
        )
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            optimizer, T_max=20, eta_min=0
        )
        return [optimizer], [scheduler]

example_model = ExampleModel()
train_NN.show_architecture(example_model)

In [ ]:
%%script echo skipping

# Set the CUDA_VISIBLE_DEVICES environment variable
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

#metrics_callback = train_NN.MetricsCallback()
logger = CSVLogger("logs", name=str(PRETRAINED_MODEL))

#trainer = pl.Trainer(max_epochs=1, callbacks=[metrics_callback], log_every_n_steps=1)
trainer = pl.Trainer(max_epochs=15, log_every_n_steps=1, logger=logger)

trainer.fit(
    model=example_model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader
)

results_df = colab_functions.get_experiment_results()
results_df.to_csv("pretrained_model_results.csv", index=False)
colab_functions.plot_experiment_results(results_df)

In [ ]:
results_df = pd.read_csv("pretrained_model_results - Copy.csv")
colab_functions.plot_experiment_results(results_df)

# Findings the best version of tf_efficientnetv2


In [ ]:
%%script echo skipping

tf_efficientnetv2_models = timm.list_models("tf_efficientnetv2*", pretrained=True)


for pretrained_model in tf_efficientnetv2_models:
    try:
        temp_model = train_NN.PretrainedModelEvaluator(pretrained_model)

        (
            preprocess_train,
            preprocess_val,
            preprocess_test,
        ) = prepare_data.get_timm_transforms(temp_model)

        (
            train_dataloader,
            val_dataloader,
            test_dataloader,
        ) = prepare_data.get_dataloaders(
            preprocess_train=preprocess_train,
            preprocess_val=preprocess_val,
            preprocess_test=preprocess_test,
        )

        # metrics_callback = train_NN.MetricsCallback()
        logger = CSVLogger("logs", name=str(pretrained_model))

        # trainer = pl.Trainer(max_epochs=1, callbacks=[metrics_callback], log_every_n_steps=1)
        trainer = pl.Trainer(max_epochs=15, log_every_n_steps=1, logger=logger)

        trainer.fit(
            model=temp_model,
            train_dataloaders=train_dataloader,
            val_dataloaders=val_dataloader,
        )

        # Clean up resources
        resources_to_delete = [
            temp_model,
            preprocess_train,
            preprocess_val,
            preprocess_test,
            train_dataloader,
            val_dataloader,
            test_dataloader,
            trainer,
        ]

        gc.collect()
    except RuntimeError as e:
        pass

results_df = colab_functions.get_experiment_results()
results_df.to_csv("tf_efficientnetv2_models_results.csv", index=False)
colab_functions.plot_experiment_results(results_df)

In [ ]:
results_df = pd.read_csv("tf_efficientnetv2_models_results.csv")
colab_functions.plot_experiment_results(results_df)

In [ ]:
(
    results_df.assign(epoch=lambda df: df.epoch.astype(int))
    .query("epoch == 14")
    .sort_values(by="val_f1", ascending=False)
)

# Finding the best learining rate

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir='/content/lightning_logs'

# Evaluate the test set


In [ ]:
preprocess_test = timm.data.create_transform(**data_cfg, is_training=False)

test_data = datasets.ImageFolder(
    root=test_dir,
    transform=preprocess_test,
    target_transform=None,
)

test_dataloader = DataLoader(
    test_data,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    drop_last=False,
    pin_memory=True,
)

model.eval()
predictions = trainer.predict(model, test_dataloader)

In [ ]:
all_labels = torch.tensor(test_dataloader.dataset.targets)
all_labels

In [ ]:
probabilities = torch.sigmoid((torch.cat(predictions, dim=0)))

# Threshold probabilities to get binary predictions (0 or 1)
threshold = 0.5
binary_predictions = (probabilities > threshold).float().view(-1)
binary_predictions

In [ ]:
acc = (all_labels == binary_predictions).sum().item() / len(all_labels)


metric_f1 = BinaryF1Score()
f1 = metric_f1(all_labels, binary_predictions)


bcm = BinaryConfusionMatrix()
bcm(all_labels, binary_predictions)
fig_, ax_ = bcm.plot()